# Credit Card PTB by TF Agent DQN

In [1]:
import os
import sys
curr_folder = globals()['_dh']

os.chdir(os.path.join(curr_folder[0], '..'))
sys.path.append(os.path.join(curr_folder[0], '..'))
sys.path.append(os.path.join(curr_folder[0], '../marketing_rl'))

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.networks.q_network import QNetwork
from tf_agents.metrics import tf_metrics
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import TFUniformReplayBuffer
from tf_agents.utils.common import element_wise_squared_loss
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, roc_auc_score
from marketing_rl.environment.biclass_tf_env import BiClassTFEnv, EnvMode

## Load Data

In [3]:
data_path = 'data/Credit Card Lead Prediction Dataset/train_s3TEQDk.csv'
df = pd.read_csv(data_path)
print('total data size is', df.shape[0])

total data size is 245725


## Preparation

### Prep Data

In [4]:
label_col = 'Is_Lead'
index_col = 'ID'
feat_cols = ['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code',
             'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df[feat_cols], df[label_col], random_state=0, test_size=0.3, shuffle=True)
print('train size is', X_train.shape[0])
print('test size is', X_test.shape[0])
print('train positive {}, train negative {}'.format(sum(y_train==1),sum(y_train==0)))

train size is 172007
test size is 73718
train positive 40818, train negative 131189


In [6]:
numer_feats = ['Age', 'Vintage', 'Avg_Account_Balance']
numer_transformer = StandardScaler()


binary_feats = ['Is_Active', 'Credit_Product']
binary_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="constant", fill_value='No')), ("ordinal", OrdinalEncoder())]
)

cat_feats = ['Gender', 'Region_Code', 'Occupation', 'Channel_Code']
cat_transformer = OneHotEncoder(handle_unknown='ignore')

transformer = ColumnTransformer(
    transformers=[
        ("num", numer_transformer, numer_feats),
        ("ordinal", binary_transformer, binary_feats),
        ("cat", cat_transformer, cat_feats)
    ], sparse_threshold = 0
)

In [7]:
X_train_t = transformer.fit_transform(X_train)
X_test_t = transformer.transform(X_test)

In [8]:
pos_neg_ratio = sum(y_train==1)/sum(y_train==0)
print('[train] positive sample to negative sample is', pos_neg_ratio)

[train] positive sample to negative sample is 0.3111388912180137


### Prep Environment

In [9]:
train_data_x = np.array(X_train_t)
train_data_y = np.array(y_train)

In [55]:
train_env = BiClassTFEnv(data_x=train_data_x, data_y=train_data_y, pos_neg_ratio=pos_neg_ratio, early_stop=10)
train_tf_env = tf_py_environment.TFPyEnvironment(train_env)

## Agent Initialization

In [69]:
# define parameters
fc_layers = (64,)
learning_rate = 1e-5
batch_size = 128

num_iterations = 10000

log_interval = 200
eval_interval = 1000

In [57]:
# q network
q_net = QNetwork(
    input_tensor_spec = train_tf_env.observation_spec(),
    action_spec = train_tf_env.action_spec(),
    fc_layer_params = fc_layers,
)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_step_counter = tf.compat.v2.Variable(0)

# dqn agent
agent = dqn_agent.DqnAgent(
    time_step_spec = train_tf_env.time_step_spec(),
    action_spec = train_tf_env.action_spec(),
    q_network = q_net,
    optimizer = optimizer,
    td_errors_loss_fn = element_wise_squared_loss,
    train_step_counter = train_step_counter,
)
agent.initialize()

## Agent Training

In [58]:
# initiate replay buffer for training
replay_buffer = TFUniformReplayBuffer(
    data_spec = agent.collect_data_spec,
    batch_size = train_tf_env.batch_size,
)
print("Batch Size: {}".format(train_env.batch_size))

replay_observer = [replay_buffer.add_batch]

# policy
eval_policy = agent.policy
collect_policy = agent.collect_policy

Batch Size: None


In [59]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls = 3, 
    sample_batch_size = batch_size,
    num_steps = 2
).prefetch(3)

date_iter = iter(dataset)

In [60]:
metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric(),
    
]

driver = dynamic_step_driver.DynamicStepDriver(
    env = train_tf_env,
    policy = collect_policy,
    observers = replay_observer + metrics,
)

In [61]:
def compute_avg_return(environment, policy, num_episodes=10):
    print('computing average return for {} episodes'.format(num_episodes))
    total_return = 0.0
    for i in range(num_episodes):
        print(f'episode {i}')
        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
            total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

In [68]:
eval_data_x = np.array(X_test_t)
eval_data_y = np.array(y_test)

eval_env = BiClassTFEnv(data_x=eval_data_x, data_y=eval_data_y, pos_neg_ratio=pos_neg_ratio, mode=EnvMode.TEST)
eval_tf_env = tf_py_environment.TFPyEnvironment(eval_env)

In [67]:
num_eval_episodes = 1

agent.train_step_counter.assign(0)
episode_len, step_len = [], []

final_time_step, policy_state = driver.run()

for i in range(num_iterations):
    final_time_step, _ = driver.run(final_time_step, policy_state)
    experience, _ = next(date_iter)
    
    train_loss = agent.train(experience=experience)
    step = agent.train_step_counter.numpy()
    
    if step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss.loss))
        episode_len.append(metrics[3].result().numpy())
        step_len.append(step)
        print('Average episode length: {}'.format(metrics[3].result().numpy()))
        
    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_tf_env, agent.policy, num_eval_episodes)
        print('step = {0}: Average Return = {1}'.format(step, avg_return))

step = 200: loss = 0.3271228075027466
Average episode length: 190.0
step = 400: loss = 0.35739457607269287
Average episode length: 276.0
step = 600: loss = 0.3588135242462158
Average episode length: 256.0
step = 800: loss = 0.35346686840057373
Average episode length: 198.0
step = 1000: loss = 0.31780490279197693
Average episode length: 188.0
computing average return for 1 episodes
episode 0
step = 1000: Average Return = 387786528.0


## Evaluation

In [45]:
avg_return = compute_avg_return(eval_tf_env, agent.policy, num_eval_episodes)
print(avg_return)

computing average return for 3 episodes
episode 0
episode 1
episode 2
20500766.0
